In [1]:
import numpy as np
from keras_facenet import FaceNet
# Specify the path to the .npz file you want to load
e = input("Enter the passing year: ")  # For example: 2025
file_path = f'C:\\Users\\pc\\Desktop\\faceattendencemodel\\models\\{e}.npz'

# Load the .npz file
data = np.load(file_path)

# Extract the variables from the loaded data using the saved keys
EMBEDDED_X= data['EMBEDDED_X']
Y = data['Y']

embedder = FaceNet()
def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

In [2]:
print(Y)

['21ESBCA003' '21ESBCA003' '21ESBCA003' ... '22ESBCS203' '22ESBCS203'
 '22ESBCS203']


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mtcnn.mtcnn import MTCNN
import cv2 as cv


encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)



X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

from sklearn.svm import SVC
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

accuracy_score(Y_train, ypreds_train)
accuracy_score(Y_test,ypreds_test)

t_im = cv.imread("C:\\Users\\pc\\Desktop\\faceattendencemodel\\data\\test\\21ESBCA003.jpg")
t_im = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)
detector = MTCNN()
x,y,w,h = detector.detect_faces(t_im)[0]['box']

t_im = t_im[y:y+h, x:x+w]
t_im = cv.resize(t_im, (160,160))
test_im = get_embedding(t_im)

test_im = [test_im]
ypreds = model.predict(test_im)
print(ypreds)
print(encoder.inverse_transform(ypreds))

import pickle
#save the model
with open('C:\\Users\\pc\\Desktop\\faceattendencemodel\\models\\svm2025.pkl','wb') as f:
    pickle.dump(model,f)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[0]
[0]
